<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c231fcd51194a3d32893acda624aa9b8d8633ab37cf710772251ad71d5f74a8a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 14:42:06
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 1.50 K (0.01%)
Current PnL: -20.55 L (-13.64%)
CY Booked + Current PnL: -9.11 L (-6.04%)
-------------------
Total profit:  1.39 L
Total loss:  -21.94 L
-------------------
Total Booked + Current PnL: 17.87 L (14.45%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.05 L (62.43%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.72,115074.0,14570.0,14534.0,-0.01,14.50,12.63,28.96,79.0,1.00,0.82,53.39,MH,ATH,METALS
14,BLUESTARCO,2080.00,10.40,51.0,H-MC,2.96,190520.0,25850.0,17471.0,0.66,15.70,9.17,26.31,89.0,1.48,1.36,24.93,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.60,39.0,M-SC,1.35,83368.0,-17409.0,17499.0,0.16,-17.27,20.99,0.09,245.0,-0.99,0.59,9.53,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.52,43.0,H-LC,3.90,207171.0,2671.0,20634.0,-0.06,1.31,9.96,11.39,10.0,0.13,1.47,10.37,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.42,205155.0,13413.0,24352.0,0.07,7.00,11.87,19.69,99.0,0.55,1.46,14.27,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,REPCOHOME,880.00,-59.02,59.0,H-SC,3.37,235211.0,-51294.0,308620.0,4.91,-17.90,131.21,89.82,134.0,-0.17,1.67,24.44,XY24,NTT,FINANCE
68,STARHEALTH,761.00,16.36,68.0,H-SC,5.87,258944.0,-22179.0,155030.0,4.04,-7.89,59.87,47.26,144.0,-0.14,1.84,38.74,XY24,NTT,INSURANCE
74,TCS,4406.34,-29.65,45.0,H-LC,12.22,280910.0,-65046.0,133292.0,2.98,-18.80,47.45,19.73,1.0,-0.49,2.00,3.46,X40,ATH,IT
75,TITAGARH,1548.00,-1.23,60.0,H-SC,4.83,210889.0,-31288.0,142055.0,2.90,-12.92,67.36,45.74,158.0,-0.22,1.50,37.16,XY24,NTT,ENGINEERING
85,WHIRLPOOL,2270.00,-46.47,34.0,M-SC,1.10,90412.0,-1086.0,79834.0,1.88,-1.19,88.30,86.07,223.0,-0.01,0.64,31.30,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HAPPSTMNDS,1488.71,-25.89,17.0,H-SC,11.91,80000.0,-48165.0,158192.0,-4.62,-37.58,197.74,85.85,132.0,-0.30,0.57,0.00,AR,ATH,IT
46,KPIGREEN,731.05,-8.64,29.0,H-SC,6.74,108751.0,-16546.0,75473.0,-2.58,-13.21,69.40,47.03,141.0,-0.22,0.77,36.17,MH,ATH,POWER
53,QUESS,986.00,-45.45,32.0,M-SC,37.81,54104.0,-10902.0,161830.0,-2.00,-16.77,299.11,232.18,198.0,-0.07,0.39,0.00,XY24,NTT,MISC
82,VALIANTORG,838.00,-300.25,37.0,H-SC,2.76,111248.0,-52357.0,173669.0,-1.98,-32.00,156.11,74.15,139.0,-0.30,0.79,42.60,XR,NTT,CHEMICALS
38,INDIGOPNTS,1408.00,100.33,35.0,M-SC,2.11,131440.0,-43119.0,43152.0,-1.96,-24.70,32.83,0.02,221.0,-1.00,0.94,14.33,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1230.14,65.0,M-SC,20.88,180942.0,-201.0,45670.0,-1.47,-0.11,25.24,25.10,235.0,-0.00,1.29,45.01,XY24,NTT,HEALTHCARE
60,SAIL,228.0,43.54,50.0,M-MC,12.34,226856.0,1894.0,165764.0,-0.66,0.84,73.07,74.53,192.0,0.01,1.62,33.37,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-21.11,48.0,H-MC,2.13,135702.0,-86.0,42841.0,-0.11,-0.06,31.57,31.48,107.0,-0.00,0.97,12.05,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,45.0,M-LC,5.71,150648.0,-297.0,109747.0,0.35,-0.20,72.85,72.51,53.0,-0.00,1.07,5.81,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,65.0,H-SC,13.51,131220.0,-11259.0,30784.0,-0.59,-7.90,23.46,13.70,163.0,-0.37,0.93,54.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.60,39.0,M-SC,1.35,83368.0,-17409.0,17499.0,0.16,-17.27,20.99,0.09,245.0,-0.99,0.59,9.53,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,51.0,H-SC,2.08,222417.0,-47250.0,83584.0,-1.54,-17.52,37.58,13.47,138.0,-0.57,1.58,13.09,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,37.0,H-SC,1.53,140507.0,-35396.0,77419.0,-0.23,-20.12,55.10,23.89,149.0,-0.46,1.00,21.35,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.38,42.0,H-MC,6.23,102239.0,-29596.0,71823.0,-0.24,-22.45,70.25,32.03,98.0,-0.41,0.73,14.52,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,50.0,M-MC,12.34,226856.0,1894.0,165764.0,-0.66,0.84,73.07,74.53,192.0,0.01,1.62,33.37,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,67.0,M-LC,6.23,213194.0,8118.0,104444.0,-0.28,3.96,48.99,54.89,52.0,0.08,1.52,29.01,X5K,ATH,METALS
14,BLUESTARCO,2080.00,10.40,51.0,H-MC,2.96,190520.0,25850.0,17471.0,0.66,15.70,9.17,26.31,89.0,1.48,1.36,24.93,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.42,205155.0,13413.0,24352.0,0.07,7.00,11.87,19.69,99.0,0.55,1.46,14.27,XY25,NTT,AC
17,CAMS,5211.76,-6.27,45.0,H-SC,1.83,106627.0,4623.0,39303.0,-0.45,4.53,36.86,43.06,122.0,0.12,0.76,22.42,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.72,115074.0,14570.0,14534.0,-0.01,14.50,12.63,28.96,79.0,1.00,0.82,53.39,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,70.0,H-MC,10.83,194680.0,14872.0,97535.0,0.43,8.27,50.10,62.52,88.0,0.15,1.39,45.18,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,67.0,M-LC,6.23,213194.0,8118.0,104444.0,-0.28,3.96,48.99,54.89,52.0,0.08,1.52,29.01,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.01,35.0,H-SC,9.35,125822.0,2486.0,129144.0,-0.08,2.02,102.64,106.72,119.0,0.02,0.90,25.86,AR,ATH,MISC
86,WIPRO,420.00,-14.25,45.0,M-LC,5.71,150648.0,-297.0,109747.0,0.35,-0.20,72.85,72.51,53.0,-0.00,1.07,5.81,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280305.0,-35537.0,148197.0,0.09,-11.25,52.87,35.67,5.0,-0.24,2.00,1.04,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.77,34.0,H-LC,5.77,207522.0,-44246.0,97452.0,0.14,-17.57,46.96,21.13,27.0,-0.45,1.48,11.62,X40,ATH,PAINTS
42,ITC,452.00,-39.61,40.0,H-LC,2.16,195502.0,-4636.0,25533.0,-0.24,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-10.67,40.0,H-LC,8.00,250900.0,-6163.0,41298.0,-1.29,-2.40,16.46,13.67,24.0,-0.15,1.79,17.52,XY25,NTT,FMCG
19,CIPLA,1662.74,-22.52,43.0,H-LC,3.90,207171.0,2671.0,20634.0,-0.06,1.31,9.96,11.39,10.0,0.13,1.47,10.37,X40N,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.03,57.0,H-LC,2.00,162400.0,-23695.0,71180.0,-0.11,-12.73,43.83,25.51,15.0,-0.33,1.16,19.68,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.16,195502.0,-4636.0,25533.0,-0.24,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.37,237442.0,-8709.0,87925.0,0.05,-3.54,37.03,32.18,16.0,-0.10,1.69,31.23,X200,ATH,IT
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,3.05,276214.0,10788.0,46017.0,-0.11,4.06,16.66,21.40,11.0,0.23,1.97,10.98,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,46.0,H-LC,3.40,214500.0,4134.0,24646.0,0.85,1.97,11.49,13.68,37.0,0.17,1.53,18.81,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280305.0,-35537.0,148197.0,0.09,-11.25,52.87,35.67,5.0,-0.24,2.00,1.04,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.16,195502.0,-4636.0,25533.0,-0.24,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
74,TCS,4406.34,-29.65,45.0,H-LC,12.22,280910.0,-65046.0,133292.0,2.98,-18.80,47.45,19.73,1.0,-0.49,2.00,3.46,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.09,311150.0,-1738.0,173435.0,-1.03,-0.56,55.74,54.87,3.0,-0.01,2.22,6.06,X40,BTT,IT
80,UNITDSPR,1644.00,-11.37,56.0,H-LC,6.88,232286.0,-1858.0,50476.0,-0.77,-0.79,21.73,20.77,86.0,-0.04,1.65,6.07,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.03,57.0,H-LC,2.00,162400.0,-23695.0,71180.0,-0.11,-12.73,43.83,25.51,15.0,-0.33,1.16,19.68,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.16,195502.0,-4636.0,25533.0,-0.24,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
19,CIPLA,1662.74,-22.52,43.0,H-LC,3.90,207171.0,2671.0,20634.0,-0.06,1.31,9.96,11.39,10.0,0.13,1.47,10.37,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.77,34.0,H-LC,5.77,207522.0,-44246.0,97452.0,0.14,-17.57,46.96,21.13,27.0,-0.45,1.48,11.62,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,46.0,H-LC,3.40,214500.0,4134.0,24646.0,0.85,1.97,11.49,13.68,37.0,0.17,1.53,18.81,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.65,45.0,H-LC,12.22,280910.0,-65046.0,133292.0,2.98,-18.80,47.45,19.73,1.0,-0.49,2.00,3.46,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.09,311150.0,-1738.0,173435.0,-1.03,-0.56,55.74,54.87,3.0,-0.01,2.22,6.06,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.16,195502.0,-4636.0,25533.0,-0.24,-2.32,13.06,10.44,4.0,-0.18,1.39,3.12,X40,NTT,FMCG
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280305.0,-35537.0,148197.0,0.09,-11.25,52.87,35.67,5.0,-0.24,2.00,1.04,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,49.0,H-LC,7.51,250896.0,-10723.0,129939.0,0.17,-4.10,51.79,45.57,7.0,-0.08,1.79,6.76,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,31.71,80256.0,-33293.0,73346.0,1.38,-29.32,91.39,35.27,268.0,-0.45,0.57,96.43,XR,NTT,HOTELS
49,MASFIN,398.61,-18.25,50.0,H-SC,11.52,93000.0,-4980.0,26579.0,0.60,-5.08,28.58,22.05,152.0,-0.19,0.66,35.18,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.72,115074.0,14570.0,14534.0,-0.01,14.50,12.63,28.96,79.0,1.00,0.82,53.39,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,65.0,H-SC,13.51,131220.0,-11259.0,30784.0,-0.59,-7.90,23.46,13.70,163.0,-0.37,0.93,54.88,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.56,146016.0,-3204.0,147389.0,1.44,-2.15,100.94,96.62,208.0,-0.02,1.04,58.02,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.56,146016.0,-3204.0,147389.0,1.44,-2.15,100.94,96.62,208.0,-0.02,1.04,58.02,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,62.0,H-SC,6.65,148451.0,-30620.0,96345.0,-1.79,-17.10,64.90,36.71,135.0,-0.32,1.06,47.29,XR,NTT,BANKS
26,GREENPANEL,537.00,233.33,60.0,M-SC,3.69,148938.0,-34140.0,113118.0,0.99,-18.65,75.95,43.14,230.0,-0.30,1.06,40.10,XY24,NTT,MISC
32,HINDZINC,730.22,31.07,67.0,M-LC,6.23,213194.0,8118.0,104444.0,-0.28,3.96,48.99,54.89,52.0,0.08,1.52,29.01,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,70.0,H-MC,10.83,194680.0,14872.0,97535.0,0.43,8.27,50.10,62.52,88.0,0.15,1.39,45.18,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.13
1,25,43.34
2,50,75.10


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.09
LC    32.18
MC    23.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.06
X40      14.58
X40N     11.20
XR       11.00
XY25     10.39
AR        8.15
OX40N     7.70
X200      1.69
MH        1.59
X5K       1.52
SR        1.14
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.99
H-LC    25.65
H-MC    20.51
M-SC    13.67
M-LC     5.52
M-MC     2.94
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-6.33,41.37
IT,12.62,-18.64,82.90
FINANCE,10.56,-13.77,68.71
MISC,6.94,-18.18,83.66
BANKS,6.37,-11.52,70.19
PAINTS,5.54,-18.65,36.33
ELECTRICAL,5.38,-9.16,47.46
INSURANCE,3.82,-3.09,41.63
AC,3.51,1.76,14.69


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3210928.0,22
XR,1365535.0,14
AR,1184291.0,9
X40,793138.0,10
X40N,662296.0,9
OX40N,564829.0,10
XY25,479523.0,7
SR,262403.0,2
X5K,104444.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3716727.0,29
M-SC,1511160.0,17
H-MC,1275173.0,15
H-LC,1193941.0,15
M-LC,374798.0,4
M-MC,370759.0,2
L-SC,260266.0,3
L-MC,60458.0,1
L-LC,42037.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196042.0      6
M-SC       XY24       850011.0      7
H-SC       AR         814316.0      5
           XR         798463.0      7
H-MC       XY24       568400.0      4
H-LC       X40        508520.0      5
M-MC       XY24       370759.0      2
H-SC       X40N       310157.0      4
           SR         262403.0      2
           OX40N      259873.0      4
H-LC       X40N       219307.0      3
H-MC       X40        211913.0      4
H-LC       AR         201119.0      2
H-MC       XY25       178136.0      2
L-SC       XR         170964.0      2
M-SC       AR         168856.0      2
M-LC       XY24       160607.0      2
M-SC       XY25       147389.0      1
           OX40N      143831.0      4
H-MC       X40N       132832.0      2
M-SC       XR         128368.0      2
H-LC       XY25       111961.0      3
M-LC       XR         109747.0      1
           X5K        104444.0      1
H-MC       XR          97535.0      1
L-SC       OX40N       89302.0      1
H-LC       X200        87925.0      1
H-SC       MH          75473.0      1
M-SC       X40         72705.0      1
H-MC       OX40N       71823.0      1
H-LC       XY24        65109.0      1
L-MC       XR          60458.0      1
L-LC       XY25        42037.0      1
H-MC       MH          14534.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
